In [167]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import torch.nn.functional as F
from torchvision.transforms import ToTensor
from torchvision import datasets
from torch.optim import Adam

In [168]:
train_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor())

test_data = datasets.FashionMNIST(
    root='data',
    train=False,
    download=True,
    transform=ToTensor())

print(len(train_data), len(test_data))



60000 10000


In [169]:
train_set = DataLoader(train_data, batch_size=10, shuffle=True)
test_set = DataLoader(test_data, batch_size=10000, shuffle=False)

In [170]:
class Net(nn.Module):
    def __init__(self) -> None:
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, 3)
        self.conv2 = nn.Conv2d(16, 64, 3)
        self.fc1 = nn.Linear(9216, 64)
        self.fc2 = nn.Linear(64, 10)
        self.softmax = nn.Softmax(dim=1)
        self.dropout1 = nn.Dropout(0.5)
        self.dropout2 = nn.Dropout(0.25)

    def forward(self, X):
        x = self.conv1(X)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)


        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)

        return output

In [171]:
model = Net()
loss = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=0.001)
num_epochs = 10


In [172]:
def test_model():
    model.eval()
    total = 0.0
    correct = 0.0
    
    with torch.no_grad():
        for data, target in test_set:
            y_pred = model(data)
            _, predicted = torch.max(y_pred, 1)
            cost = loss(y_pred , target)
            total += target.size(0)
            correct += (predicted ==target).sum().item()

    accuracy = round(correct / total * 100, 2)

    return cost , accuracy

In [173]:
for i in range(num_epochs):
    model.train()
    for batch_idx, (data, target) in enumerate(train_set):

        y_pred = model(data)

        cost = loss(y_pred , target)

        cost.backward()
        optimizer.step()
        optimizer.zero_grad()

        
    test_cost , accuracy = test_model()
    print(f'{int((i+1)*10)}% done -- Test Cost: {round(float(test_cost),5)} -- Test Accuracy: {accuracy}%')
    


10% done -- Test Cost: 0.32812 -- Test Accuracy: 88.4%
20% done -- Test Cost: 0.28436 -- Test Accuracy: 89.73%
30% done -- Test Cost: 0.25531 -- Test Accuracy: 90.75%
40% done -- Test Cost: 0.27221 -- Test Accuracy: 90.1%
50% done -- Test Cost: 0.24238 -- Test Accuracy: 91.26%
60% done -- Test Cost: 0.23522 -- Test Accuracy: 91.4%
70% done -- Test Cost: 0.23701 -- Test Accuracy: 91.43%
80% done -- Test Cost: 0.23192 -- Test Accuracy: 91.81%
90% done -- Test Cost: 0.23748 -- Test Accuracy: 91.94%
100% done -- Test Cost: 0.243 -- Test Accuracy: 91.65%


In [174]:

test_model()

(tensor(0.2430), 91.65)